# Fetching news articles from "News API" 

In [1]:
import requests
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

In [2]:
# generate a API Key from newsapi website :  https://newsapi.org/
api_key="1cc968acf6c94176b3ee0d6b76b05c04"

In [3]:
# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [4]:
def news_sentiment_analysis(stock,ticker):
   # Function to fetch news articles
    def fetch_news(api_key, query, max_results=100):
        url = f'https://newsapi.org/v2/everything?q={query}&pageSize={max_results}&apiKey={api_key}'
        response = requests.get(url)
        data = response.json()
        articles = data.get('articles', [])
        return articles 
    
    # Fetch news articles for Stock
    query = stock + " , " + ticker
    articles = fetch_news(api_key, query)
    
    # Extract datetime, content
    news_data = [{'datetime': article.get('publishedAt', ''),
                  'content': article.get('content', '')} for article in articles]
    
    # Create a DataFrame
    news_df = pd.DataFrame(news_data)
    
    # Remove articles with empty content
    news_df = news_df[(news_df['content'].str.lower() != '[removed]') & (news_df['content'] != '')]
    news_df=news_df.reset_index()
    
    # Function to calculate sentiment scores
    def get_sentiment_scores(text):
        if not text:
            return 0
        scores = analyzer.polarity_scores(text)
        return scores['compound']

    #compound_score=scores['compound']
    # Function to classify sentiment
    def classify_sentiment(score):
        if score >= 0.05:
            return 'Positive'
        elif score <= -0.05:
            return 'Negative'
        else:
            return 'Neutral'
    
    # Calculate sentiment scores and classify sentiment
    news_df['sentiment_score'] = news_df['content'].apply(get_sentiment_scores)
    news_df['sentiment'] = news_df['sentiment_score'].apply(classify_sentiment)

    
    return news_data,news_df

In [5]:
def Visualize_News(news_data,news_df):
    # Word Cloud for all articles
    all_text = ' '.join(news_df['content'].dropna())
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('Word Cloud for All Articles')
    plt.show()
    
    # Bar Chart for Classified Sentiments
    plt.figure(figsize=(8, 6))
    sns.countplot(x='sentiment', data=news_df, palette='viridis', hue='sentiment', legend = False)
    plt.title('Bar Chart of Classified Sentiments')
    plt.xlabel('Sentiment')
    plt.ylabel('Count')
    plt.show()
    
    # Pie Chart for Sentiment Distribution
    sentiment_counts = news_df['sentiment'].value_counts()
    plt.figure(figsize=(8, 8))
    plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('viridis', len(sentiment_counts)))
    plt.title('Pie Chart of Sentiment Distribution')
    plt.show()